In [96]:
import lmdb
from bigearthnet_patch_interface.s2_interface import BigEarthNet_S2_Patch
from pathlib import Path
import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import Tensor, cat, stack
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from sklearn.metrics import accuracy_score


#### Data Loading

In [2]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

# Get paths
load_dotenv('./.env')

BEN_LMDB_PATH = os.environ.get("BEN_LMDB_PATH")
BEN_PATH = os.environ.get("BEN_PATH")

TRAIN_CSV_FILE = os.environ.get("TRAIN_CSV")
TEST_CSV_FILE = os.environ.get("TEST_CSV")
VAL_CSV_FILE = os.environ.get("VAL_CSV")

assert Path(BEN_PATH).exists()
assert Path(BEN_LMDB_PATH).exists()
assert os.path.exists(TRAIN_CSV_FILE)

In [3]:
env = lmdb.open(BEN_LMDB_PATH, readonly=True, readahead=False, lock=False)
txn = env.begin()
cur = txn.cursor()

In [73]:
import dataset_class as ds_class
importlib.reload(ds_class)

val_ds = ds_class.BenDataset(VAL_CSV_FILE, BEN_LMDB_PATH, 50)
test_ds = ds_class.BenDataset(TEST_CSV_FILE, BEN_LMDB_PATH, 50)
train_ds = ds_class.BenDataset(TRAIN_CSV_FILE, BEN_LMDB_PATH, 100)

val_loader = DataLoader(val_ds, batch_size=5, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=5, shuffle=True)
train_loader = DataLoader(train_ds, batch_size=5, shuffle=True)

# y = val_ds.__getitem__(0)
# print(y[0].shape, y[1].shape)
# print(y[1], val_ds.get_mlb().inverse_transform(np.asarray([y[1]])))

In [74]:
print(len(val_ds), len(train_loader))

for X, y in [next(iter(train_loader))]:
    print(X.shape)
    print(y.shape)
    
    break

50 20
torch.Size([5, 10, 120, 120])
torch.Size([5, 19])


#### Training

In [114]:
import conv_mixer as conv_mixer
importlib.reload(conv_mixer)


n_classes = 19
model = conv_mixer.ConvMixer(10, 256, 8, n_classes=n_classes)
if torch.cuda.is_available():
    model = model.cuda()

loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [115]:
for e in range(50):
    
    train_loss = 0.0
    cumul_acc = 0.0
    
    for inputs, labels in train_loader:
        
        # Transfer to GPU if available
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
         
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        
        # keep track of accuracy in this epoch
        outputs_sig = torch.sigmoid(outputs)
        predict = torch.round(outputs_sig)

        for i in range(5):
            cumul_acc += accuracy_score(labels.detach().numpy()[i], predict.detach().numpy()[i])
        
        # print(labels)
        # print(predict)
        
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # keep track of loss
        train_loss += loss.item()
        
        # break
        
    
    train_acc = cumul_acc / len(train_ds)
    
    print(f'Epoch {e+1}:')
    print(f'train_loss: {train_loss / len(train_loader)}')
    print(f'train_acc: {train_acc}')
    print()
     
     
print('Finished Training')

Epoch 1:
train_loss: 0.6976366072893143
train_acc: 0.49736842105263146

Epoch 2:
train_loss: 0.6927116423845291
train_acc: 0.5178947368421053

Epoch 3:
train_loss: 0.6862026542425156
train_acc: 0.5657894736842105

Epoch 4:
train_loss: 0.6833735406398773
train_acc: 0.5684210526315792

Epoch 5:
train_loss: 0.6763222277164459
train_acc: 0.6073684210526313

Epoch 6:
train_loss: 0.6720433890819549
train_acc: 0.6163157894736842

Epoch 7:
train_loss: 0.6684554070234299
train_acc: 0.6342105263157892

Epoch 8:
train_loss: 0.6629581332206727
train_acc: 0.6363157894736841

Epoch 9:
train_loss: 0.6603371888399124
train_acc: 0.669473684210526

Epoch 10:
train_loss: 0.6558683842420578
train_acc: 0.6752631578947367

Epoch 11:
train_loss: 0.6517573833465576
train_acc: 0.7194736842105265

Epoch 12:
train_loss: 0.647162738442421
train_acc: 0.7347368421052631

Epoch 13:
train_loss: 0.6439383506774903
train_acc: 0.7447368421052631

Epoch 14:
train_loss: 0.6415725290775299
train_acc: 0.7578947368421052

Ep